In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load dataset
df = pd.read_csv("Data.csv")
df = df[['aX', 'aY', 'aZ', "Result"]]

# Filter earthquake and non-earthquake samples
earthquake = df[df["Result"] == 1].iloc[:28600].reset_index(drop=True)
no_earthquake = df[df["Result"] == 0].iloc[:28600].reset_index(drop=True)

# Split features for earthquake and no earthquake
earthquake_X = earthquake[['aX', 'aY', 'aZ']].values
no_earthquake_X = no_earthquake[['aX', 'aY', 'aZ']].values

# Concatenate both classes into the final dataset
X = np.concatenate((earthquake_X, no_earthquake_X))
y = np.concatenate((np.ones(len(earthquake_X)), np.zeros(len(no_earthquake_X))))

# Normalize the data
X = X.astype('float32')
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)  # Standardize

# Reshape into windows of 100 timesteps, each with 3 features (aX, aY, aZ)
def create_dataset(data, window_size=100):
    X = []
    for i in range(len(data) - window_size + 1):
        X.append(data[i:i + window_size])  # Extract 100 timesteps
    return np.array(X)

# Create windows of 100 timesteps for both earthquake and non-earthquake data
X = create_dataset(X)

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y[:len(X)], test_size=0.2, random_state=42)

# Reshape the data to match the BiLSTM input shape (samples, timesteps, features)
# Each sample has 100 timesteps and 3 features (aX, aY, aZ)
X_train = X_train.reshape((X_train.shape[0], 100, 3))
X_test = X_test.reshape((X_test.shape[0], 100, 3))

# Convert labels to categorical (one-hot encoding)
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Define BiLSTM model
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=False, dropout=0.3), input_shape=(100, 3)),  # 100 timesteps, 3 features
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')  # Binary classification (earthquake or not)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=1,
    batch_size=32,
    verbose=1
)

# Save the trained model
model.save("model/bilstm_model.h5")

# Optionally evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1428/1428 ━━━━━━━━━━━━━━━━━━━━ 80s 56ms/step - accuracy: 0.9787 - loss: 0.0848 - val_accuracy: 0.9946 - val_loss: 0.0258


357/357 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9947 - loss: 0.0219
Test Loss: 0.02579214982688427, Test Accuracy: 0.9945713877677917
